In [1]:
import os
import locale
import math

import pandas as pd
import numpy as np

from datetime import datetime
from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from antecedentes import create_antecedentes_doc

import pypdf
from pathlib import Path

from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate

from docx2pdf import convert

/home/eduardo/Dev/ccd/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import subprocess

def generate_pdf(doc_path, path):

    subprocess.call(['soffice',
                  '--headless',
                 '--convert-to',
                 'pdf',
                 '--outdir',
                 path,
                 doc_path])
    return doc_path

In [3]:
def get_file_path(row):
    return Path('/media/informacoes_pdf/') / row['setor'].strip() / row['arquivo']


In [4]:
def get_pdf_text(row):
    arquivo = get_file_path(row)
    print(f'File {arquivo} to pypdf text')
    try:
        pdf = pypdf.PdfReader(arquivo)
        text = []
        for page in pdf.pages:
            text.append(page.extract_text())
        return ' '.join(text)
    except FileNotFoundError:
        print(f'File not found: {arquivo}')
        return ''

In [5]:
antecedentes_sql = ''' 

SELECT DISTINCT pro.numero_processo,
       pro.ano_processo,
       pro.assunto,
       pro.interessado,
       pm.Descricao as marcador      
       
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp ON gp.IdPessoa = edp.IDPessoa
WHERE pro.setor_atual = 'CCD'
AND lower(pm.Descricao) like '%antecedente%'
'''
processos_antecedentes = pd.read_sql(antecedentes_sql, conn)

/tmp/ipykernel_128126/146702693.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_antecedentes = pd.read_sql(antecedentes_sql, conn)


In [6]:
processos_antecedentes

,numero_processo,ano_processo,assunto,interessado,marcador
0,003427,2023,"APURAÇÃO DE RESPONSABILIDADE, REFERENTE AO PRO...",PREFEITURA MUNICIPAL DE JUCURUTU ...,Antecedentes
1,004304,2024,CONTAS DO CHEFE DO PODER EXECUTIVO DA PREFEITU...,PREFEITURA MUNICIPAL DE CANGUARETAMA ...,Antecedentes
2,004702,2024,CONTAS DO CHEFE DO PODER EXECUTIVO DA PREFEITU...,PREFEITURA MUNICIPAL DE BODÓ ...,Antecedentes
3,200063,2023,INADIMPLÊNCIA NA REMESSA DE SIAI DP NOVO ...,PAULO CEZAR FLORENCIO DE SOUZA ...,Antecedentes


In [7]:
sql_info_antecedentes_gaana = ''' 
WITH RankedData AS (
    SELECT 
    	ai.setor,
        concat(pro.numero_processo, '/', pro.ano_processo) as processo,
        pro.assunto,
        pro.interessado,
        concat(rtrim(ai.setor),'_',
        ai.numero_processo ,'_',
        ai.ano_processo,'_',RIGHT(concat('0000',ai.ordem),4),'.pdf') as arquivo,
        ai.datapublicacao, 
        ppe.SequencialProcessoEvento, 
        ai.numero_processo, 
        ai.ano_processo,
        ROW_NUMBER() OVER (
            PARTITION BY CONCAT(ai.numero_processo, '/', ai.ano_processo)
            ORDER BY ppe.SequencialProcessoEvento DESC
        ) AS rn
    FROM processo.dbo.vw_Ata_Informacao ai 
    LEFT JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON ai.idInformacao = ppe.IdInformacao
    LEFT JOIN processo.dbo.Processos pro
    	ON pro.numero_processo = ai.numero_processo AND pro.ano_processo = ai.ano_processo
    WHERE CONCAT(ai.numero_processo, '/', ai.ano_processo) IN (
        SELECT CONCAT(pro.numero_processo, '/', pro.ano_processo)
        FROM processo.dbo.Processos pro
        INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
            ON pmp.IdProcesso = pro.IdProcesso
        INNER JOIN processo.dbo.Pro_Marcador pm 
            ON pmp.IdMarcador = pm.IdMarcador
        LEFT JOIN processo.dbo.Exe_Debito ed 
            ON ed.IdProcessoExecucao = pro.IdProcesso
        LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
            ON edp.IDDebito = ed.IdDebito
        LEFT JOIN processo.dbo.GenPessoa gp 
            ON gp.IdPessoa = edp.IDPessoa
        WHERE pro.setor_atual = 'CCD'
        AND lower(pm.Descricao) LIKE '%antecedente%'
    )
    AND ai.setor LIKE '%GAANA%'
)
SELECT * 
FROM RankedData 
WHERE rn = 1
ORDER BY numero_processo, ano_processo;


'''
df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)

/tmp/ipykernel_128126/3946558724.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_info_gaana = pd.read_sql(sql_info_antecedentes_gaana, conn)


In [8]:
def get_transito(nome):
    with open("consultas/processos_transito_cpf.sql") as f:
        query = f.read()
    with get_connection() as conn:
        q = query.format(nome=nome)
        df = pd.read_sql(q, conn)
        df['valor_original'] = df['valor_original'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df['valor_atualizado'] = df['valor_atualizado'].apply(lambda x: locale.currency(x, grouping=True, symbol=False) if x and not math.isnan(x) else '-')
        df.fillna('', inplace=True)
        return df

def get_cpf_responsavel(nome_pessoa, numero_processo, ano_processo):
    query = f'''
SELECT gp.Documento as cpf
	FROM Processo.dbo.Processos p
		INNER JOIN Processo.dbo.Pro_ProcessosResponsavelDespesa pprd 
		ON pprd.IdProcesso = p.IdProcesso
		INNER JOIN Processo.dbo.GenPessoa gp 
		ON gp.IdPessoa = pprd.IdPessoa
	WHERE p.numero_processo = '{numero_processo}'
	AND p.ano_processo = '{ano_processo}'
	AND lower(gp.Nome) = '{nome_pessoa.lower()}'
 '''
    with get_connection() as conn:
        df = pd.read_sql(query, conn)
        if len(df) > 1:
            raise ValueError(f'Mais de um CPF para {nome_pessoa}')
        elif len(df) == 0:
            return ''
        return df['cpf'].values[0]
    
def get_valores_pessoa(row):
    pessoa_strc = row['pessoas']
    print(f'Pessoas: {pessoa_strc.pessoas}')
    valores = []
    for pessoa in pessoa_strc.pessoas:
        nome_pessoa = pessoa.nome_pessoa
        transitos = get_transito(nome_pessoa)
        cpfs = transitos['cpf'].unique()
        if len(cpfs) > 1:
            raise ValueError(f'Mais de um {cpf} por pessoa')
        elif len(cpfs) == 0:
            cpf = get_cpf_responsavel(nome_pessoa, row['numero_processo'], row['ano_processo'])
        else:
            cpf = cpfs[0]
        valores.append({'nome': nome_pessoa, 'cpf': cpf, 'transitos': transitos.to_dict(orient='records')})
    return valores

def is_vazio(valores):
    return all([not bool(v['transitos']) for v in valores])
    

In [9]:
df_info_gaana['texto_pdf'] = df_info_gaana.apply(get_pdf_text, axis=1)

File /media/informacoes_pdf/GAANA/GAANA_003427_2023_0036.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004304_2024_0019.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_004702_2024_0015.pdf to pypdf text
File /media/informacoes_pdf/GAANA/GAANA_200063_2023_0024.pdf to pypdf text


In [10]:
llm = AzureChatOpenAI(model_name="gpt-4o")

from typing import List

from pydantic import BaseModel, Field

class PessoaAntecedentes(BaseModel):
  """Pessoa cujo antecedente deve ser buscado"""
  nome_pessoa: str = Field(description="Nome da pessoa para buscar antecedentes")

class PessoasAntecedentes(BaseModel):
  """Lista de pessoas cujos antecedentes devem ser buscados"""
  pessoas: List[PessoaAntecedentes] = Field(description="Lista de pessoas para buscar antecedentes")
  
def get_pessoas_llm(texto_despacho):

  prompt = PromptTemplate.from_template("""
    Você é um agente que identifica listas de pessoas em requisições de antecedentes. 
      Você recebeu uma requisição para buscar antecedentes de pessoas em um processo. Remova qualquer vocativo do nome da pessoa, tal como Sr., Sra., Dr., Dra., etc.
      Não inclua nomes de conselheiros, advogados ou partes do processo. Ex: Sra. Conselheira Substituta Ana
Paula de Oliveira Gomes
      O texto da requisição é o seguinte:
      "{input}"
      Encontre os antecedentes das pessoas listadas na requisição.

    Sua resposta:
    """)

  structured_llm = llm.with_structured_output(schema=PessoasAntecedentes)
  chain = prompt | structured_llm
  return chain.invoke(texto_despacho)

In [11]:
df_info_gaana['pessoas'] = df_info_gaana['texto_pdf'].apply(get_pessoas_llm)

In [12]:
df_info_gaana['valores'] = df_info_gaana.apply(get_valores_pessoa, axis=1)

Pessoas: [PessoaAntecedentes(nome_pessoa='George Retlen Costa Queiroz')]


/tmp/ipykernel_128126/1674572162.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_128126/1674572162.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/tmp/ipykernel_128126/1674572162.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)


Pessoas: [PessoaAntecedentes(nome_pessoa='João Wilson de Andrade Ribeiro Filho')]


/tmp/ipykernel_128126/1674572162.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/tmp/ipykernel_128126/1674572162.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)


Pessoas: [PessoaAntecedentes(nome_pessoa='Marcelo Mário Porto Filho')]
Pessoas: [PessoaAntecedentes(nome_pessoa='Paulo Cezar Florencio de Souza')]


/tmp/ipykernel_128126/1674572162.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(q, conn)
/tmp/ipykernel_128126/1674572162.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [13]:
def valores_vazios(valores):
    return all([not bool(v['transitos']) for v in valores])

In [14]:
#df_info_gaana['len_valores'] = df_info_gaana['valores'].apply(lambda x: len(x))
df_info_gaana['sem_valores'] = df_info_gaana['valores'].apply(valores_vazios)
antecedentes = df_info_gaana

In [15]:
for row in antecedentes.itertuples():
    doc = DocxTemplate("templates/antecedentes.docx")
    #valores = [v for v in row.valores if len(v['transitos']) > 0]
    nomes = [v['nome'] for v in row.valores]
    if len(nomes) > 1:
        responsaveis = ', '.join(nomes[:-1]) + ' e ' + nomes[-1]
    else:
        responsaveis = nomes[0] if nomes else ''
    for v in row.valores:
        if v['cpf'] is None:
            v['cpf'] = ''
    context = {
        'processo': row.processo,
        'assunto': row.assunto,
        'interessado': row.interessado,
        'valores': row.valores,
        'responsaveis': responsaveis,
        'data': datetime.now().strftime("%d/%m/%Y"),
        'sem_valores': row.sem_valores
    }
    processo_underline = row.processo.replace('/', '_')

    doc.render(context)
    doc.save(f"saidas/antecedentes/{processo_underline}.docx")
    generate_pdf(f"saidas/antecedentes/{processo_underline}.docx", f"saidas/antecedentes/")
    print(f"Gerado arquivo para o processo {row.processo}")

convert /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/003427_2023.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/003427_2023.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 003427/2023


convert /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/004304_2024.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/004304_2024.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 004304/2024


convert /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/004702_2024.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/004702_2024.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 004702/2024


convert /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/200063_2023.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/antecedentes/200063_2023.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 200063/2023
